In [1]:
from DataLoader import Image_Dataset
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import utils
from torchvision import transforms

In [2]:
transform = transforms.Compose([transforms.Resize((150,150), interpolation=2), transforms.ToTensor(), transforms.Normalize(0.5,0.5)])
train_directory_in_str = "111880_269359_upload_seg_train/seg_train"
train_set_list, classes = Image_Dataset.load_images(train_directory_in_str)
train_set = Image_Dataset(train_set_list, train_directory_in_str, transform)
train_set_list

[('glacier/5155.jpg', tensor(0)),
 ('glacier/14540.jpg', tensor(0)),
 ('glacier/17287.jpg', tensor(0)),
 ('glacier/10172.jpg', tensor(0)),
 ('glacier/10690.jpg', tensor(0)),
 ('glacier/3354.jpg', tensor(0)),
 ('glacier/13231.jpg', tensor(0)),
 ('glacier/12120.jpg', tensor(0)),
 ('glacier/2262.jpg', tensor(0)),
 ('glacier/5268.jpg', tensor(0)),
 ('glacier/11994.jpg', tensor(0)),
 ('glacier/13998.jpg', tensor(0)),
 ('glacier/6430.jpg', tensor(0)),
 ('glacier/16307.jpg', tensor(0)),
 ('glacier/8084.jpg', tensor(0)),
 ('glacier/15209.jpg', tensor(0)),
 ('glacier/5209.jpg', tensor(0)),
 ('glacier/8521.jpg', tensor(0)),
 ('glacier/6378.jpg', tensor(0)),
 ('glacier/12224.jpg', tensor(0)),
 ('glacier/6463.jpg', tensor(0)),
 ('glacier/8793.jpg', tensor(0)),
 ('glacier/9478.jpg', tensor(0)),
 ('glacier/5135.jpg', tensor(0)),
 ('glacier/18899.jpg', tensor(0)),
 ('glacier/2768.jpg', tensor(0)),
 ('glacier/14072.jpg', tensor(0)),
 ('glacier/9958.jpg', tensor(0)),
 ('glacier/19319.jpg', tensor(0)),


In [3]:
test_directory_in_str = "seg_test"
test_set_list, classes = Image_Dataset.load_images(test_directory_in_str, classes)
test_set = Image_Dataset(test_set_list, test_directory_in_str, transform)

In [4]:
test_set[0]

(tensor([[[ 0.5451,  0.5373,  0.5059,  ...,  0.0745,  0.0510,  0.0510],
          [ 0.5294,  0.5373,  0.5059,  ...,  0.0902,  0.0588,  0.0510],
          [ 0.5373,  0.5529,  0.5137,  ...,  0.0745,  0.0431,  0.0353],
          ...,
          [ 0.9451,  0.9373,  0.9216,  ..., -0.9686, -0.9529, -0.9373],
          [ 0.9529,  0.9451,  0.9294,  ..., -1.0000, -0.9922, -0.9765],
          [ 0.9137,  0.9294,  0.9373,  ..., -1.0000, -1.0000, -1.0000]],
 
         [[ 0.7098,  0.7020,  0.6706,  ...,  0.3961,  0.3961,  0.3882],
          [ 0.6863,  0.6941,  0.6627,  ...,  0.4118,  0.4039,  0.3882],
          [ 0.6706,  0.6863,  0.6627,  ...,  0.3961,  0.3882,  0.3725],
          ...,
          [ 0.9373,  0.9294,  0.9137,  ..., -0.7412, -0.7255, -0.7255],
          [ 0.9451,  0.9373,  0.9216,  ..., -0.7961, -0.7804, -0.7647],
          [ 0.9059,  0.9216,  0.9294,  ..., -0.8039, -0.8118, -0.8196]],
 
         [[ 0.9216,  0.9137,  0.8824,  ...,  0.6314,  0.6235,  0.6392],
          [ 0.8980,  0.9059,

In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
use_cuda

True

In [6]:
trainloader = torch.utils.data.DataLoader(train_set, batch_size=10, shuffle=True)
dataiter = iter(trainloader)
images, labels = dataiter.next()

conv1 = nn.Conv2d(3, 16, 5, padding=2)
conv2 = nn.Conv2d(16, 32, 5, padding=2)
conv3 = nn.Conv2d(32, 64, 5, padding=2)
pool = nn.MaxPool2d(3, 3)

features = nn.Sequential(
            nn.Conv2d(3, 5, 3, padding=1), 
            nn.ReLU(inplace=True),
            nn.Conv2d(5, 10, 3, padding=1), 
            nn.MaxPool2d(3, 3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(10),
            nn.Conv2d(10, 20, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(20),
            nn.Conv2d(20, 40, 3, padding=1),
            nn.MaxPool2d(3, 3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(40))



print(images.shape)

x = features(images)
print(x.shape)

torch.Size([10, 3, 150, 150])
torch.Size([10, 40, 16, 16])


In [124]:
class ConvNeuralNet(torch.nn.Module):
    def __init__(self, classes): 
        super(ConvNeuralNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 5, 3, padding=1), 
            nn.ReLU(inplace=True),
            nn.Conv2d(5, 10, 3, padding=1), 
            nn.MaxPool2d(3, 3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(10),
            nn.Conv2d(10, 20, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(20),
            nn.Conv2d(20, 40, 3, padding=1),
            nn.MaxPool2d(3, 3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(40))

        self.fcc = nn.Sequential(
            nn.Linear(40*16*16, 1000),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(1000, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 100),
            nn.Dropout(p=0.5),
            nn.ReLU(inplace=True),
            nn.Linear(100, 50),
            nn.ReLU(inplace=True),
            nn.Linear(50, len(classes)))
            
    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1,40*16*16)
        x = self.fcc(x)
        return x
    
    def predict(self, x):
        x = self.conv(x)
        x = x.view(-1,40*16*16)
        x = self.fcc(x)
        return x

In [125]:
batch_size = 100
epochs = 50
lr_rate = 0.01
train_loader = utils.data.DataLoader(train_set, batch_size = batch_size, shuffle = True)
model = ConvNeuralNet(classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.2)

In [126]:
n_total_steps = len(train_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [200, 3, 150, 150] = 200, 3, 22500
        
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 20 == 0:
            print (f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
    scheduler.step()
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch [1/50], Step [20/141], Loss: 1.7780
Epoch [1/50], Step [40/141], Loss: 1.7926
Epoch [1/50], Step [60/141], Loss: 1.7714
Epoch [1/50], Step [80/141], Loss: 1.7585
Epoch [1/50], Step [100/141], Loss: 1.7354
Epoch [1/50], Step [120/141], Loss: 1.7318
Epoch [1/50], Step [140/141], Loss: 1.6822
Epoch [2/50], Step [20/141], Loss: 1.5649
Epoch [2/50], Step [40/141], Loss: 1.5239
Epoch [2/50], Step [60/141], Loss: 1.4188
Epoch [2/50], Step [80/141], Loss: 1.3497
Epoch [2/50], Step [100/141], Loss: 1.3681
Epoch [2/50], Step [120/141], Loss: 1.4176
Epoch [2/50], Step [140/141], Loss: 1.3530
Epoch [3/50], Step [20/141], Loss: 1.3269
Epoch [3/50], Step [40/141], Loss: 1.2491
Epoch [3/50], Step [60/141], Loss: 1.2244
Epoch [3/50], Step [80/141], Loss: 1.1698
Epoch [3/50], Step [100/141], Loss: 1.1364
Epoch [3/50], Step [120/141], Loss: 1.1026
Epoch [3/50], Step [140/141], Loss: 1.2142
Epoch [4/50], Step [20/141], Loss: 1.0304
Epoch [4/50], Step [40/141], Loss: 0.9975
Epoch [4/50], Step [60/14

Epoch [28/50], Step [60/141], Loss: 0.3251
Epoch [28/50], Step [80/141], Loss: 0.2393
Epoch [28/50], Step [100/141], Loss: 0.2934
Epoch [28/50], Step [120/141], Loss: 0.2504
Epoch [28/50], Step [140/141], Loss: 0.2330
Epoch [29/50], Step [20/141], Loss: 0.2774
Epoch [29/50], Step [40/141], Loss: 0.2935
Epoch [29/50], Step [60/141], Loss: 0.2676
Epoch [29/50], Step [80/141], Loss: 0.2732
Epoch [29/50], Step [100/141], Loss: 0.2790
Epoch [29/50], Step [120/141], Loss: 0.2809
Epoch [29/50], Step [140/141], Loss: 0.2580
Epoch [30/50], Step [20/141], Loss: 0.1850
Epoch [30/50], Step [40/141], Loss: 0.2633
Epoch [30/50], Step [60/141], Loss: 0.3043
Epoch [30/50], Step [80/141], Loss: 0.2665
Epoch [30/50], Step [100/141], Loss: 0.3027
Epoch [30/50], Step [120/141], Loss: 0.3569
Epoch [30/50], Step [140/141], Loss: 0.1781
Epoch [31/50], Step [20/141], Loss: 0.2422
Epoch [31/50], Step [40/141], Loss: 0.2334
Epoch [31/50], Step [60/141], Loss: 0.2007
Epoch [31/50], Step [80/141], Loss: 0.2504
Ep

KeyboardInterrupt: 

In [127]:
test_loader = utils.data.DataLoader(test_set, batch_size = batch_size, shuffle = False)

In [128]:
with torch.no_grad():
    model.eval()
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for _, i in enumerate(classes)]
    n_class_samples = [0 for _, i in enumerate(classes)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        tmp, predicted = torch.max(outputs, 1)
       
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i, label in  enumerate(classes):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {label}: {acc} %')

Accuracy of the network: 85.66666666666667 %
Accuracy of glacier: 78.11934900542495 %
Accuracy of sea: 86.07843137254902 %
Accuracy of mountain: 82.66666666666667 %
Accuracy of street: 88.42315369261478 %
Accuracy of forest: 95.56962025316456 %
Accuracy of buildings: 84.43935926773456 %
